In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.init as init
from torch.autograd import Variable

import os
import time
import numpy as np
from PIL import Image
from utils.dataloader import *
#use AUC for AUC and CI, auc2 for precision, AUC and CI, auc3 precision auc and CI
from utils.auc import *
from utils import new_transforms
import argparse
import random

In [24]:
ngpu = 1
nc = 3
imgSize = 299

step_freq = 20000000


root_dir = '/gpfs/data/abl/deepomics/tsirigoslab/histopathology/Tiles/LungTilesSorted/'
num_classes = 3
tile_dict_path = '/gpfs/data/abl/deepomics/tsirigoslab/histopathology/Tiles/Lung_FileMappingDict.p'


In [25]:
manualSeed = random.randint(1, 10000) # fix seed

random.seed(manualSeed)
torch.manual_seed(manualSeed)

cudnn.benchmark = True

In [26]:
# Random data augmentation
augment = transforms.Compose([new_transforms.Resize((imgSize, imgSize)),
                              transforms.RandomHorizontalFlip(),
                              new_transforms.RandomRotate(),
                              new_transforms.ColorJitter(0.25, 0.25, 0.25, 0.05),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose([new_transforms.Resize((imgSize,imgSize)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data = {}
loaders = {}

for dset_type in ['train', 'valid']:
    if dset_type == 'train' and True:
        data[dset_type] = TissueData(root_dir, dset_type, transform = augment, metadata=False, test_valid=False)
    else:
        data[dset_type] = TissueData(root_dir, dset_type, transform = transform, metadata=False, test_valid=False)

    loaders[dset_type] = torch.utils.data.DataLoader(data[dset_type], batch_size=32, shuffle=True)
    print('Finished loading %s dataset: %s samples' % (dset_type, len(data[dset_type])))

class_to_idx = data['train'].class_to_idx
classes = data['train'].classes

print('Class encoding:')
print(class_to_idx)

Loading from: Solid_Tissue_Normal
number of samples: 73015
Loading from: TCGA-LUSC
number of samples: 204134
Loading from: TCGA-LUAD
number of samples: 186044
Finished loading train dataset: 463193 samples
Loading from: Solid_Tissue_Normal
number of samples: 13542
Loading from: TCGA-LUSC
number of samples: 50047
Loading from: TCGA-LUAD
number of samples: 41918
Finished loading valid dataset: 105507 samples
Class encoding:
{'Solid_Tissue_Normal': 0, 'TCGA-LUAD': 1, 'TCGA-LUSC': 2}


In [28]:
print(classes)

['Solid_Tissue_Normal', 'TCGA-LUAD', 'TCGA-LUSC']


In [29]:
# Custom weights initialization

def init_model(model):
    for m in model.modules():
        if isinstance(m,nn.Conv2d):
            m.weight.data = init.xavier_normal(m.weight.data)
        elif isinstance(m,nn.BatchNorm2d):
            m.weight.data.normal_(-0.1, 0.1)

class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, pool, **kwargs):
        super(BasicConv2d, self).__init__()

        self.pool = pool
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

        self.relu = nn.ReLU()

        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = self.conv(x)

        if self.pool:
            x = F.max_pool2d(x, 2)
        
        x = self.relu(x)
        x = self.bn(x)
        x = self.dropout(x)
        return x

# Define model
class cancer_CNN(nn.Module):
    def __init__(self, nc, imgSize, ngpu):
        super(cancer_CNN, self).__init__()
        self.nc = nc
        self.imgSize = imgSize
        self.ngpu = ngpu
        #self.data = opt.data
        self.conv1 = BasicConv2d(nc, 16, False, kernel_size=5, padding=1, stride=2, bias=True)
        self.conv2 = BasicConv2d(16, 32, False, kernel_size=3, bias=True)
        self.conv3 = BasicConv2d(32, 64, True, kernel_size=3, padding=1, bias=True)
        self.conv4 = BasicConv2d(64, 64, True, kernel_size=3, padding=1, bias=True)
        self.conv5 = BasicConv2d(64, 128, True, kernel_size=3, padding=1, bias=True)
        self.conv6 = BasicConv2d(128, 64, True, kernel_size=3, padding=1, bias=True)
        self.linear = nn.Linear(5184, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

###############################################################################

# Create model objects
model = cancer_CNN(nc, imgSize, ngpu)
init_model(model)
model.train()

criterion = nn.CrossEntropyLoss()

# Load checkpoint models if needed


model.cuda()

# Set up optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))


/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  


In [30]:
def get_tile_probability(tile_path):

    """
    Returns an array of probabilities for each class given a tile
    @param tile_path: Filepath to the tile
    @return: A ndarray of class probabilities for that tile
    """

    # Some tiles are empty with no path, return nan
    if tile_path == '':
        return np.full(num_classes, np.nan)

    tile_path = root_dir + tile_path

    with open(tile_path, 'rb') as f:
        with Image.open(f) as img:
            img = img.convert('RGB')

    # Model expects a 4D tensor, unsqueeze first dimension
    img = transform(img).unsqueeze(0)

    img = img.cuda()

    # Turn output into probabilities with softmax
    var_img = Variable(img, volatile=True)
    output = F.softmax(model(var_img)).data.squeeze(0)

    return output.cpu().numpy()

# Load tile dictionary

with open(tile_dict_path, 'rb') as f:
    tile_dict = pickle.load(f)

def aggregate(file_list, method):

    """
    Given a list of files, return scores for each class according to the
    method and labels for those files.
    @param file_list: A list of file paths to do predictions on
    @param method: 'average' - returns the average probability score across
                               all tiles for that file
                   'max' - predicts each tile to be the class of the maximum
                           score, and returns the proportion of tiles for
                           each class
    @return: a ndarray of class probabilities for all files in the list
             a ndarray of the labels
    """

    model.eval()
    predictions = []
    true_labels = []

    for file in file_list:
        tile_paths, label = tile_dict[file]

        folder = classes[label]

        def add_folder(tile_path):
            if tile_path == '':
                return ''
            else:
                return folder + '/' + tile_path

        # Add the folder for the class name in front
        add_folder_v = np.vectorize(add_folder)
        tile_paths = add_folder_v(tile_paths)

        # Get the probability array for the file
        prob_v = np.vectorize(get_tile_probability, otypes=[np.ndarray])
        probabilities = prob_v(tile_paths)


        """
        imgSize = probabilities.shape()
        newShape = (imgSize[0], imgSize[1], 3)
        probabilities = np.reshape(np.stack(probabilities.flat), newShape)
        """

        if method == 'average':
            probabilities = np.stack(probabilities.flat)
            prediction = np.nanmean(probabilities, axis = 0)

        elif method == 'max':
            probabilities = np.stack(probabilities.flat)
            probabilities = probabilities[~np.isnan(probabilities).all(axis=1)]
            votes = np.nanargmax(probabilities, axis=1)
            
            out = np.array([sum(votes == i) for i in range(num_classes)])
            prediction = out / out.sum()

        else:
            raise ValueError('Method not valid')

        predictions.append(prediction)
        true_labels.append(label)

    return np.array(predictions), np.array(true_labels)

###############################################################################

def early_stop(val_history, t=3, required_progress=0.0001):

    """
    Stop the training if there is no non-trivial progress in k steps
    @param val_history: a list contains all the historical validation auc
    @param required_progress: the next auc should be higher than the previous by 
        at least required_progress amount to be non-trivial
    @param t: number of training steps 
    @return: a boolean indicates if the model should early stop
    """
    
    if (len(val_history) > t+1):
        differences = []
        for x in range(1, t+1):
            differences.append(val_history[-x]-val_history[-(x+1)])
        differences = [y < required_progress for y in differences]
        if sum(differences) == t: 
            return True
        else:
            return False
    else:
        return False


stop_training = False




In [42]:
best_AUC = 0.0

print('Starting training')
start = time.time()
local_time = time.ctime(start)
print(local_time)

print(time.time())
for epoch in range(1,25):
    data_iter = iter(loaders['train'])
    print(data_iter)
    i = 0
    
    print(len(loaders['train']))
    
    while i < len(loaders['train']):
        model.train()
        img, label = data_iter.next()
        i += 1

        # Drop the last batch if it's not the same size as the batchsize
        if img.size(0) != 32:
            break

        img = img.cuda()
        label = label.cuda()

        input_img = Variable(img)
        target_label = Variable(label)

        train_loss = criterion(model(input_img), target_label)
        print(train_loss)
        # Zero gradients then backward pass
        optimizer.zero_grad()
        train_loss.backward()
      
        correc=0
        total=0

        optimizer.step()
        
        print('[%d/%d][%d/%d] Training Loss: %f'
               % (epoch, 25, i, len(loaders['train']), train_loss.item()))
        ii=i+((epoch)*len(loaders['train']))
        #get validation AUC every step_freq 
        if ii % step_freq == 0:
            val_predictions, val_labels = aggregate(data['valid'].filenames, method='average')

            data_ = np.column_stack((data['valid'].filenames,np.asarray(val_predictions),np.asarray(val_labels)))
            #data_.dump(open('{0}/outputs/val_pred_label_avg_step_{1}.npy'.format(1,str(ii)), 'wb'))
            #torch.save(model.state_dict(), '{0}/checkpoints/step_{1}.pth'.format(1, str(ii)))           
            #print('validation scores:')

            #roc_auc = get_auc('{0}/images/val_roc_step_{1}.jpg'.format(opt.experiment,epoch), val_predictions, val_labels, classes = range(num_classes))
            for k, v in roc_auc.items(): 
                if k in range(num_classes):
                    k = classes[k] 
                #experiment.log_metric("{0} AUC".format(k), v)
                print('%s AUC: %0.4f' % (k, v))

    #save the checkpoint at every epoch
    #torch.save(model.state_dict(), '{0}/checkpoints/epoch_{1}.pth'.format(opt.experiment, str(epoch)))

    #print(time.time())
    # Get validation AUC once per epoch
    




# Final evaluation
print('Finished training, best AUC: %0.4f' % (best_AUC))
end = time.time()
print(end-start)

Starting training
Wed Aug 14 17:53:13 2019
1565819593.9164448
14475
tensor(0.9032, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][1/14475] Training Loss: 0.903194
tensor(1.0196, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][2/14475] Training Loss: 1.019568
tensor(0.8736, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][3/14475] Training Loss: 0.873595
tensor(0.9566, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][4/14475] Training Loss: 0.956565
tensor(0.9175, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][5/14475] Training Loss: 0.917497
tensor(0.9562, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][6/14475] Training Loss: 0.956221
tensor(1.0273, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][7/14475] Training Loss: 1.027304
tensor(0.8861, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][8/14475] Training Loss: 0.886058
tensor(1.0820, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][9/14475] Training Loss: 1.082000
tensor(1.0198, device='cuda:0', grad_fn=<

tensor(0.8541, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][82/14475] Training Loss: 0.854140
tensor(0.8161, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][83/14475] Training Loss: 0.816101
tensor(0.8269, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][84/14475] Training Loss: 0.826914
tensor(1.0192, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][85/14475] Training Loss: 1.019155
tensor(1.0190, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][86/14475] Training Loss: 1.018974
tensor(0.8885, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][87/14475] Training Loss: 0.888496
tensor(0.9259, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][88/14475] Training Loss: 0.925857
tensor(0.9098, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][89/14475] Training Loss: 0.909754
tensor(0.8718, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][90/14475] Training Loss: 0.871757
tensor(1.0185, device='cuda:0', grad_fn=<NllLossBackward>)
[1/25][91/14475] Training Loss: 1.018530


KeyboardInterrupt: 

In [43]:
import multiprocessing

multiprocessing.cpu_count()

40

In [44]:
import threading
threading.activeCount()

5

In [ ]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--experiment', default='', help="name of experiment to test")
#parser.add_argument('--model', default='', help="name of model to test")
#parser.add_argument('--root_dir', type=str, default='<ROOT_PATH><CANCER_TYPE>TilesSorted/', help='Data directory .../dataTilesSorted/')
#parser.add_argument('--num_class', type=int, default=2, help='number of classes ')
#parser.add_argument('--tile_dict_path', type=str, default='"<ROOT_PATH><CANCER_TYPE>_FileMappingDict.p', help='Tile dictinory path')
#parser.add_argument('--val', type=str, default='test', help='validation set')

#opt = parser.parse_args()


#test_val = str(opt.val)

imgSize = 299

transform = transforms.Compose([new_transforms.Resize((imgSize,imgSize)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_data = TissueData(root_dir, 'test', transform = transform, metadata=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

classes = test_data.classes
class_to_idx = test_data.class_to_idx